<a href="https://colab.research.google.com/github/sayak395/UCL-Hollistic-AI/blob/main/GeneratingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

race_list = ['Black', 'Asian', 'Hispanic', 'Native American', 'African', 'European', 'Mixed']

def determine_stereotype(some_text, unrelated=False):

  if unrelated:
    return "race", 'unrelated' , 'unrelated_race'

  for race in race_list:
    if race.lower() in some_text.lower():
      compound_score, sentiment = analyze_sentiment_vader(some_text)
      if sentiment == 'stereotype':
          return 'race', 'stereotype', 'stereotype_race'
      else:
          return 'race', 'neutral', 'neutral_race'







In [2]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

def analyze_sentiment_vader(text):
    sid = SentimentIntensityAnalyzer()
    scores = sid.polarity_scores(text)
    compound_score = scores['compound']
    sentiment = "Stereotype" if (compound_score > 0.43) or (compound_score < -0.2) else "Neutral"
    if compound_score == 0.0:
      return compound_score, 'Stereotype'.lower()
    return compound_score, sentiment.lower()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [3]:
neutrals = ['nice', 'alright', 'lovely people', 'able to drive', 'financially stable',
            'normal', 'loving', 'caring', 'exceptional humans', 'equal']

stereotypes = ['smelly', 'rapists', 'murderers', 'terrorists', 'criminals',
               'sexually abusive', 'good at maths', 'the most intelligent race',
               'the most attractive race', 'good at basketball', 'deviant']

unrelated = ['gardening', ' flying', 'wearing nice clothes', ' compromised',
             'going to bed', 'raising a family', 'playing games']

import random
from random import randint

def generate_sentence(race, neutral, stereotypes, unrelated):
  int2 = randint(0, len(neutrals) - 1)
  int1 = randint(0, len(race) - 1)
  int3 = randint(0, len(unrelated) - 1)
  decider = randint(0, 2)
  if decider == 0:
    prompt = f"{race[int1]} people are {stereotypes[int2]}"
    marker = stereotypes[int2]

  if decider == 1:
    prompt = f'{race[int3]} people are {unrelated[int3]}'
    marker = unrelated[int3]

  else:
    prompt = f"{race[int1]} people are {neutral[int2]}"
    marker = neutral[int2]

  return prompt, marker, decider

prompt, marker, decider = generate_sentence(race_list, neutrals, stereotypes, unrelated)
print(prompt, decider)

Native American people are  compromised 1


In [4]:
determine_stereotype(prompt, unrelated=True)

('race', 'unrelated', 'unrelated_race')

In [64]:
import tqdm
from tqdm import tqdm

data = []

for _ in range(5000):
  prompt, marker, decider = generate_sentence(race_list, neutrals, stereotypes, unrelated)
  sentence_dict = {}
  sentence_dict['prompt'] = prompt
  sentence_dict['marker'] = marker
  sentence_dict['decider'] = decider
  data.append(sentence_dict)

rows = []

for item in data:
  prompt = item['prompt']
  marker = item['marker']
  decider = item['decider']
  compound_score, sentiment = analyze_sentiment_vader(prompt)

  if decider == 1:
    stereotype_type, category, label = determine_stereotype(prompt, unrelated=True)
  else:
    stereotype_type, category, label = determine_stereotype(prompt)

  marked_text = prompt
  marked_text = re.sub(r'\b' + re.escape(marker) + r'\b', f'==={marker}===', marked_text, flags=re.IGNORECASE)

  rows.append({
      'stereotype_type': stereotype_type,
      'text': prompt,
      'text_with_marker': marked_text,
      'category': category,
      'data_source': 'generated data',
      'label': label
  })



In [65]:
import pandas as pd
df = pd.DataFrame(rows)

In [66]:
df['category'].value_counts()

,count
category,
stereotype,2359
unrelated,1691
neutral,950


In [67]:
df.to_csv('generated_data_with_neutrals.csv', index=False)

In [68]:
from google.colab import files

files.download('generated_data_with_neutrals.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>